
# **Highway to Hell (Or Heaven)**
---

**By Ashvin Anilkumar**              

## 1. Introduction(20%)

In a fiercely competitive marketplace, two manufacturing plants, **ShoeCo** and **BootCo**, race to deliver the same product to customers but utilize different production technologies, leading to distinct cost structures and defect rates. These plants feed a shared logistics network comprising ports, carriers, warehouses, and final retail shops. The market is competitive, enforcing a uniform selling price regardless of origin.

The core challenge is to determine the optimal production quantities at each plant and the subsequent flow of goods through the network to **maximize total expected profit**. This decision must be robust against uncertain market demand, which is modeled using a discrete set of scenarios (e.g., Low, Medium, High demand).

|Scenario ($\omega$) |Probability ($\pi_\omega$) |Demand ($D_{s\omega}$) |
|-------------|-----------------|---------------|
|Low Demand  |30%  | 0.7$D_{s\omega}$ |  
|Medium Demand  |50%  |$D_{s\omega}$|
|High Demand  |20%  |1.3$D_{s\omega}$|



<img height=400 align="center" src="https://github.com/ashvin-a/Final-Project-524/blob/main/docs/Logistic-graph.svg?raw=1"/>






This is a simple diagram of our optimisation problem. We will start with this and later on build the actual figure of the problem.
The problem is formulated as a Two-Stage Stochastic Mixed-Integer Linear Program (MILP):

- Stage 1 (Strategic/Tactical): Production quantities, initial shipping from plants to ports, and carrier selection must be decided before demand is known.

- Stage 2 (Operational/Recourse): Once demand is realized (per scenario), inventory allocation, final distribution to shops, shortage penalties, and defect handling are determined.

## 2. Approach (15%)




We will first analyse the data from the excel sheet and then we will try to make a model which will solve for the optimal path and quantity to be produced for both the plants. Let's start by analysing the data that is given.


In [33]:
import pandas as pd

sheet_names = ["OrderList",
               "FreightRates",
               "WhCosts",
               "WhCapacities",
               "ProductsPerPlant",
               "PlantPorts",
               "VmiCustomers",]

df = pd.read_excel("data.xlsx", sheet_name=sheet_names)

# Data Analysis
for index, dataframe in df.items():
  print(f"\n\n\nTable name: {index}\n Dataframe Summary : {dataframe.columns}\n")
  for column in dataframe.columns:
    if column not in ["Order ID", "Product ID", "minimum cost", "max_wgh_qty", "minm_wgh_qty"]:
      # print(f"Column name : {column}\n ")
      print(f"Column name : {column}\n Unique Entries :{dataframe[column].unique()}")




Table name: OrderList
 Dataframe Summary : Index(['Order ID', 'Order Date', 'Origin Port', 'Carrier', 'TPT',
       'Service Level', 'Ship ahead day count', 'Ship Late Day count',
       'Customer', 'Product ID', 'Plant Code', 'Destination Port',
       'Unit quantity', 'Weight'],
      dtype='object')

Column name : Order Date
 Unique Entries :<DatetimeArray>
['2013-05-26 00:00:00']
Length: 1, dtype: datetime64[ns]
Column name : Origin Port
 Unique Entries :['PORT09' 'PORT04' 'PORT05']
Column name : Carrier
 Unique Entries :['V44_3' 'V444_0' 'V444_1']
Column name : TPT
 Unique Entries :[1 4 0 2 3]
Column name : Service Level
 Unique Entries :['CRF' 'DTP' 'DTD']
Column name : Ship ahead day count
 Unique Entries :[3 0 5 6 1]
Column name : Ship Late Day count
 Unique Entries :[0 6 1 3]
Column name : Customer
 Unique Entries :['V55555_53' 'V555555555555555_29' 'V5555555_30' 'V55555_2'
 'V555555555555_31' 'V55555555_32' 'V5555555_12' 'V55555_4' 'V555555_6'
 'V5555555555_1' 'V5555_33' '

#### 2.1 Data Analysis
As we can observe, the excel sheet consists of 7 sheets. Here is the description on what each sheets tells us about:

- OrderList: Contains historical records of how the orders were routed and demand satisfied.
- FreightRates: Describes all available couriers, the weight gaps for each individual lane and rates associated.
- WhCosts: Specifies the cost associated in storing the products in given warehouse measured in dollars per unit.
- WhCapacities: Lists warehouse capacities measured in number of orders per day and the WhCosts specifies the cost associated in storing the products in given warehouse measured in dollars per unit.
- ProductsPerPlant: Lists all supported warehouse-product combinations.
- PlantPorts:  Describes the allowed links between the warehouses and shipping ports in real world.
- VmiCustomers: Lists all special cases, where warehouse is only allowed to support specific customer, while any other non-listed warehouse can supply any customer.

We can see that there is no separate entity for plants, which makes this a perfect dataset since we have the control over all the parameters of each plant, making this model more versatile.

#### 2.2 Key Entities and Sets
The following players will be considered as the sets for our optimisation problem:
- Plants ($i \in I$): Two plants.
  - Characteristics: Production capacity, unit production cost, defect rate.
  - ShoeCo: Low cost, High defect rate.
  - BootCo: High cost, Low defect rate.
- Ports ($p \in P$): Intermediate transit hubs.
  - Characteristics: Handling capacity, throughput cost.
- Carriers ($k \in K$): Transport options from Ports to Warehouses.     
  - Characteristics: Shipping cost, delivery speed (Fast vs. Slow)
- Warehouses ($w \in W$): Storage facilities.
  - Characteristics: Storage capacity, holding cost, safety regulation (zero tolerance for defective goods storage).
- Shops ($s \in S$): Final retail destinations.
  - Characteristics: Uncertain customer demand.
  - Scenarios ($\omega \in \Omega$): Possible states of the world (e.g., Demand High, Medium, Low) with probability $\pi_\omega$


#### 2.3 Variables  
**First-Stage Variables (Here-and-Now)**:
- $x_{i}$: Quantity produced at plant $i$.
- $y_{ip}$: Quantity shipped from plant $i$ to port $p$.
- $z_{pwk} \in \{0, 1\}$ (or Integer): Selection of carrier $k$ to transport goods from port $p$ to warehouse $w$.

**Second-Stage Variables (Recourse - Scenario Dependent)**:
- $f_{p w k, \omega}$: Quantity of good product shipped from port $p$ to warehouse $w$ via carrier $k$ in scenario $\omega$.
- $d_{w, \omega}$: Quantity of defective product identified and disposed of/returned at warehouse $w$.
- $I_{w, \omega}$: Inventory level of good product at warehouse $w$ in scenario $\omega$.
- $v_{ws, \omega}$: Quantity shipped from warehouse $w$ to shop $s$ in scenario $\omega$.
- $L_{s, \omega}$: Lost sales (unmet demand) at shop $s$ in scenario $\omega$.

####2.4 Parameters and Costs
- Production: $C^{prod}_i$ (Cost), $R^{defect}_i$ (Defect Rate), $Cap^{plant}_i$ (Capacity).
- Logistics: $C^{trans}_{ip}$ (Plant-Port), $C^{carrier}_{pwk}$ (Port-Warehouse), $C^{final}_{ws}$ (Warehouse-Shop).
- Speed/Market: $\alpha_k$ (Market share bonus factor for fast carriers).
- Inventory: $C^{hold}_w$ (Holding cost per unit), $Req^{safe}_w = 1.4$ (Safety stock ratio: 140% of demand).
- Market: $P^{sell}$ (Selling Price), $D_{s, \omega}$ (Demand at shop $s$ in scenario $\omega$).
- Penalties: $C^{short}$ (Cost per unit of unmet demand).


####2.5 Constraints
**Stage 1 Constraints**:
  1. Production Capacity: $x_i \le Cap^{plant}_i$.
  2. Plant Outflow: $\sum_p y_{ip} = x_i$.

**Stage 2 Constraints** (for each scenario $\omega$):

  3.  Defect Sorting (Entry to Warehouse):
The flow arriving at warehouses must be stripped of defects.

$$\text{Total Volume from } i = y_{ip}$$

$$\text{Good volume} = y_{ip} (1 - R^{defect}_i)$$

$$\sum_{w,k} f_{p w k, \omega} \le \sum_i y_{ip} (1 - R^{defect}_i)$$


4.  Warehouse Balance:
Inflow from ports = Outflow to shops + Inventory.
$\sum_{p,k} f_{p w k, \omega} = \sum_s v_{ws, \omega} + I_{w, \omega}$.
5.  Safety Stock Regulation:
Warehouse inventory must cover 140% of the goods sent to connected shops.
$I_{w, \omega} \ge 1.40 \sum_s v_{ws, \omega}$.
<!-- (Note: This is a strict constraint that might force higher production or limit sales). -->
6.  Demand Satisfaction:
$v_{ws, \omega} \le D_{s, \omega}$.
Total Sales at $s$: $\sum_w v_{ws, \omega} + L_{s, \omega} = D_{s, \omega}$.
7.  Defect Handling: Defective items ($x_i \cdot R^{defect}_i$) incur a disposal/handling cost but cannot enter warehouse inventory.

#### 2.6 Objective Function

Maximize:$$\sum_{\omega \in \Omega} \pi_\omega \left[ \text{Revenue}_\omega - (\text{Stage 2 Costs}_\omega) \right] - (\text{Stage 1 Costs})$$
- Stage 1 Costs: Production + Initial Transport + Carrier Selection Fixed Costs.
- Stage 2 Costs: Warehouse-to-Shop Transport + Inventory Holding + Shortage Penalties + Defect Disposal.
- Revenue: $P^{sell} \times (\text{Total Sold})$. Optional: Add bonus revenue for "Fast" carrier usage modeling the early market entry advantage.


Now, we understand the way in which we need to optimise this problem. Before using the actual data, we can start with a simple data that is somewhat realistic to see if our approach is working or not.

## The Freaking Solution (30%)


In [ ]:
!pip install gamspy
!gamspy install license 2a571695-fbd3-4aa6-9587-b3d6083d1be8

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.0/223.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 31.9 MB/s eta 0:00:00
╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /usr/local/lib/python3.12/dist-packages/gamspy/_cli/install.py:71 in license │
│                                                                              │
│    68 │   is_alp = not os.path.isfile(license)                               │
│    69 │                                                                      │
│    70 │   if is_alp and len(license) != 36:                                  │
│ ❱  71 │   │   raise ValidationError(                                         │
│    72 │   │   │   f"Access code is a 36 character string or an absolute path │
│    73 │   │   │   f"license file but {len(license)} character string ({licen │
│    74 │   │   )                  

In [ ]:
# Data Preprocessing (10%)
import gamspy as gp
import pandas as pd
import numpy as np

In [ ]:
gp.set_options({'USE_PY_VAR_NAME': 'yes'})
# Do our implementations here
m = gp.Container()

# -------------------------------------------------------------------------
# 1. SETS
# -------------------------------------------------------------------------
# Physical Locations
i = gp.Set(m, name="i", records=["ShoeCo", "BootCo"], description="Plants")
p = gp.Set(m, name="p", records=["PortA", "PortB"], description="Ports")
w = gp.Set(m, name="w", records=["WH1", "WH2", "WH3"], description="Warehouses")
s = gp.Set(m, name="s", records=["Shop1", "Shop2", "Shop3", "Shop4"], description="Shops")

# Options
k = gp.Set(m, name="k", records=["Fast", "Slow"], description="Carrier Types")

# Stochasticity
omega = gp.Set(m, name="omega", records=["Low", "Medium", "High"], description="Demand Scenarios")

# -------------------------------------------------------------------------
# 2. PARAMETERS
# -------------------------------------------------------------------------

# --- Plant Data ---
# Plant 1: Low Cost, High Defect. Plant 2: High Cost, Low Defect
prod_cost = gp.Parameter(m, name="prod_cost", domain=[i], records=pd.DataFrame([
    ["Plant1", 10.0], ["Plant2", 15.0]], columns=["i", "value"]))

defect_rate = gp.Parameter(m, name="defect_rate", domain=[i], records=pd.DataFrame([
    ["Plant1", 0.10], ["Plant2", 0.02]], columns=["i", "value"]))

capacity = gp.Parameter(m, name="capacity", domain=[i], records=pd.DataFrame([
    ["Plant1", 5000], ["Plant2", 4000]], columns=["i", "value"]))

# --- Logistics Costs ---
trans_cost_ip = gp.Parameter(m, name="trans_cost_ip", domain=[i, p], records=2.0) # Simplified uniform cost

# Carrier Costs (Fast is more expensive)
carrier_cost = gp.Parameter(m, name="carrier_cost", domain=[p, w, k])
# Set slow to base, fast to higher
carrier_cost.setRecords([
    ["PortA", "WH1", "Slow", 3.0], ["PortA", "WH1", "Fast", 6.0],
    # ... populate full matrix or use logic. Setting scalar for brevity in example
])
# Fallback scalar load for example if specific records missing
# In practice, load full dataframe

# Fixed cost for selecting a carrier route (Stage 1 decision cost)
carrier_fixed_cost = gp.Parameter(m, name="carrier_fixed_cost", records=1000.0)

final_dist_cost = gp.Parameter(m, name="final_dist_cost", domain=[w, s], records=1.5)

# --- Scenario Data ---
prob = gp.Parameter(m, name="prob", domain=[omega], records=pd.DataFrame([
    ["Low", 0.3], ["Medium", 0.5], ["High", 0.2]], columns=["omega", "value"]))

# Demand (scenario dependent)
demand = gp.Parameter(m, name="demand", domain=[s, omega])
# Generate some dummy demand data
import itertools
dem_data = []
base_dem = {"Shop1": 100, "Shop2": 150, "Shop3": 200, "Shop4": 120}
factors = {"Low": 0.7, "Medium": 1.0, "High": 1.3}
for sh, sc in itertools.product(s.records.values.flatten(), omega.records.values.flatten()):
    dem_data.append([sh, sc, base_dem[sh] * factors[sc]])
demand.setRecords(pd.DataFrame(dem_data, columns=["s", "omega", "value"]))

# --- Economic Parameters ---
price = gp.Parameter(m, name="price", records=50.0)
holding_cost = gp.Parameter(m, name="holding_cost", domain=[w], records=0.5)
shortage_penalty = gp.Parameter(m, name="shortage_penalty", records=60.0) # High penalty for lost sales
defect_penalty = gp.Parameter(m, name="defect_penalty", records=5.0) # Cost to dispose

# Market Share / Speed Bonus (Optional revenue booster for using Fast carrier)
# Let's say Fast carrier adds a small premium to revenue or reduces lost sales
# Modeled here as simply a cost trade-off, but could be added to obj.

# -------------------------------------------------------------------------
# 3. VARIABLES
# -------------------------------------------------------------------------

# --- Stage 1 (Here-and-Now) ---
x_prod = gp.Variable(m, name="x_prod", domain=[i], type="Positive", description="Production Qty")
y_ship = gp.Variable(m, name="y_ship", domain=[i, p], type="Positive", description="Plant to Port Flow")

# Binary decision: Do we contract carrier k for route p->w?
z_select = gp.Variable(m, name="z_select", domain=[p, w, k], type="Binary", description="Carrier Selection")

# --- Stage 2 (Recourse) ---
# Flow of GOOD product
f_flow = gp.Variable(m, name="f_flow", domain=[p, w, k, omega], type="Positive", description="Port to WH Flow")

# Inventory at Warehouse
inv = gp.Variable(m, name="inv", domain=[w, omega], type="Positive", description="Ending Inventory")

# Flow to Shops
v_sales = gp.Variable(m, name="v_sales", domain=[w, s, omega], type="Positive", description="WH to Shop Flow")

# Unmet Demand
l_lost = gp.Variable(m, name="l_lost", domain=[s, omega], type="Positive", description="Lost Sales")

# Total Profit Variable
profit = gp.Variable(m, name="profit", type="Free")

# -------------------------------------------------------------------------
# 4. EQUATIONS
# -------------------------------------------------------------------------

# --- Stage 1 Constraints ---

# 1. Production Capacity
eq_cap = gp.Equation(m, name="eq_cap", domain=[i])
eq_cap[i] = x_prod[i] <= capacity[i]

# 2. Plant Balance (Production = Outflow to Ports)
eq_plant_bal = gp.Equation(m, name="eq_plant_bal", domain=[i])
eq_plant_bal[i] = x_prod[i] == gp.Sum(p, y_ship[i, p])

# --- Stage 2 Constraints (Scenario Dependent) ---

# 3. Defect Sorting / Port Balance (Linking Stage 1 to Stage 2)
# The sum of flow leaving ports to WHs cannot exceed Good Product arrived at ports.
# Note: We need to sum over plants to get total arrival at port p.
# Total Good Arrival at p = Sum(i, y_ship[i,p] * (1 - defect_rate[i]))
eq_port_bal = gp.Equation(m, name="eq_port_bal", domain=[p, omega])

eq_port_bal[p, omega] = (
    gp.Sum([w, k], f_flow[p, w, k, omega]) <=
    gp.Sum(i, y_ship[i, p] * (1 - defect_rate[i]))
)

# 4. Carrier Selection Logical Constraint
# Flow f_flow can only exist if carrier is selected in Stage 1
# Big-M constraint. M can be Total Capacity.
M = 10000
eq_carrier_logic = gp.Equation(m, name="eq_carrier_logic", domain=[p, w, k, omega])
eq_carrier_logic[p, w, k, omega] = f_flow[p, w, k, omega] <= M * z_select[p, w, k]

# 5. Warehouse Balance
# Inflow from Ports = Outflow to Shops + Inventory
eq_wh_bal = gp.Equation(m, name="eq_wh_bal", domain=[w, omega])
eq_wh_bal[w, omega] = (
    gp.Sum([p, k], f_flow[p, w, k, omega]) ==
    gp.Sum(s, v_sales[w, s, omega]) + inv[w, omega]
)

# 6. Safety Regulation (The 140% Rule)
# Inventory must be >= 1.4 * Total Sales supported by this warehouse
# Wait, problem says "140% of good products available in that shop".
# Interpretation: 140% of what is sent to the shop? Or 140% of demand?
# Let's use: Inventory at W >= 1.4 * (Flow from W to all S)
eq_safety = gp.Equation(m, name="eq_safety", domain=[w, omega])
eq_safety[w, omega] = inv[w, omega] >= 1.4 * gp.Sum(s, v_sales[w, s, omega])

# 7. Demand Satisfaction
# Sales + Lost = Demand
eq_demand = gp.Equation(m, name="eq_demand", domain=[s, omega])
eq_demand[s, omega] = gp.Sum(w, v_sales[w, s, omega]) + l_lost[s, omega] == demand[s, omega]

# --- Objective Function ---

# Costs
# Stage 1 Cost: Production + Transport to Port + Carrier Fixed Costs
stage1_cost = (
    gp.Sum(i, prod_cost[i] * x_prod[i]) +
    gp.Sum([i, p], trans_cost_ip[i, p] * y_ship[i, p]) +
    gp.Sum([p, w, k], carrier_fixed_cost * z_select[p, w, k])
)

# Stage 2 Expected Revenue and Costs
# Revenue = Price * Sold (Note: Sold = Demand - Lost)
# Costs = Carrier Transport + WH to Shop Transport + Holding + Shortage + Defect Penalty

# Helper for Defect Cost: Total Produced * Rate * Penalty
defect_cost_total = gp.Sum(i, x_prod[i] * defect_rate[i] * defect_penalty)

# Note on Carrier Cost: Ideally this should be parameter carrier_cost[p,w,k].
# I used scalar 3.0/6.0 in setRecords earlier, assuming parameter exists.

stage2_expected_profit = gp.Sum(omega, prob[omega] * (
    # Revenue
    (price * gp.Sum(s, (demand[s, omega] - l_lost[s, omega]))) -
    # Variable Transport Costs
    (gp.Sum([p, w, k], carrier_cost[p, w, k] * f_flow[p, w, k, omega]) +
      gp.Sum([w, s], final_dist_cost[w, s] * v_sales[w, s, omega])) -
    # Holding Costs
    gp.Sum(w, holding_cost[w] * inv[w, omega]) -
    # Shortage Penalty
    gp.Sum(s, shortage_penalty * l_lost[s, omega])
))

eq_obj = gp.Equation(m, name="eq_obj")
eq_obj[...] = profit == stage2_expected_profit - stage1_cost - defect_cost_total

# -------------------------------------------------------------------------
# 5. SOLVE
# -------------------------------------------------------------------------

model = gp.Model(
    m,
    name="LogiOpt_Stochastic",
    equations=m.getEquations(),
    problem="MIP",
    sense="MAX",
    objective=profit
)

model.solve()

In [ ]:
# Sensitivity Analysis, Visualization (10%)

## Conclusion of the Project

## Extensions of the project
